In [ ]:
API_KEY = ""
MODELS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/models"
COMPLETIONS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/chat/completions"
CLARIN_BASE_URL = "https://services.clarin-pl.eu/api/v1/oapi"

In [2]:
from datasets import load_dataset
import requests

In [4]:
dataset = load_dataset("bible_para", lang1="en", lang2="pl")

In [3]:
def get_models():
    headers = {
        'Authorization': f'Bearer {API_KEY}',  
        'Content-Type': 'application/json'
    }
    response = requests.get(MODELS_ENDPOINT, headers=headers)  
    if response.status_code == 200:
        return response.json() 

In [4]:
models = get_models()
print(models)

{'data': [{'id': 'bielik', 'full_name': 'speakleash/Bielik-11B-v2.2-Instruct', 'name': 'speakleash/Bielik-11B-v2.2-Instruct', 'type': 'chat'}, {'id': 'cohere', 'full_name': 'CohereForAI/c4ai-command-r-plus', 'name': 'CohereForAI/c4ai-command-r-plus', 'type': 'chat'}, {'id': 'llama3.1', 'full_name': 'meta-llama/Meta-Llama-3.1-70B-Instruct', 'name': 'meta-llama/Meta-Llama-3.1-70B-Instruct', 'type': 'chat'}, {'id': 'llama-guard', 'full_name': 'meta-llama/Llama-Guard-3-8B', 'name': 'meta-llama/Llama-Guard-3-8B', 'type': 'chat'}, {'id': 'llama3.1-8b', 'full_name': 'meta-llama/Llama-3.1-8B-Instruct', 'name': 'meta-llama/Llama-3.1-8B-Instruct', 'type': 'chat'}, {'id': 'openchat', 'full_name': 'openchat/openchat-3.5-1210', 'name': 'openchat/openchat-3.5-1210', 'type': 'chat'}, {'id': 'llama3.3', 'full_name': 'meta-llama/Llama-3.3-70B-Instruct', 'name': 'meta-llama/Llama-3.3-70B-Instruct', 'type': 'chat'}, {'id': 'mixtral-8x22B', 'full_name': 'mistralai/Mixtral-8x22B-Instruct-v0.1', 'name': 'mi

In [12]:
def prompt_chat(model_id, prompt):
    url = COMPLETIONS_ENDPOINT
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    messages = []
    messages.append({"role": "user", "content": prompt})
    data = {
        "model": model_id,
        "messages": messages,
        #"max_tokens": max_tokens
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"CLARIN API Error: {response.status_code} - {response.text}")

In [13]:
def prompt_chat_custom_temperature(model_id, prompt, temperature = 0.5):
    url = COMPLETIONS_ENDPOINT
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    messages = []
    messages.append({"role": "user", "content": prompt})
    data = {
        "model": model_id,
        "messages": messages,
        "temperature": temperature
        #"max_tokens": max_tokens
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"CLARIN API Error: {response.status_code} - {response.text}")

In [8]:
def few_shot_translation(model_id, prompt, number_of_examples, styling=True, max_tokens=150):
    url = COMPLETIONS_ENDPOINT
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}

    messages = [{"role": "system", "content": "You are a helpful assistant that translates English to Polish. You are also required to translate the English sentence in a specific style by learning from examples provided to you. After translating, don't say anything, just only translation"}]
    if styling:
        for i in range(number_of_examples):
            messages.append({"role": "user", "content": f"Translate to Polish: '{dataset['train']['translation'][i]['en']}'"})
            messages.append({"role": "assistant", "content": f"{dataset['train']['translation'][i]['pl']}"})
    messages.append({"role": "user", "content": f"Translate the following sentence to Polish and don't add anything else: '{prompt}'"})
    data = {
        "model": model_id,
        "messages": messages
        #"max_tokens": max_tokens
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"CLARIN API Error: {response.status_code} - {response.text}")

In [9]:
for i in range(40, 45):
    en = dataset['train']['translation'][i]['en']
    pl = dataset['train']['translation'][i]['pl']

    print(en)
    print(pl)

    unstyled_response = few_shot_translation('bielik', en, 20, False)
    styled_response = few_shot_translation('bielik', en, 20, True)

    print("")
    print(f"unstyled response: {unstyled_response}")
    print(f"styled response: {styled_response}")

    print("")
    print("========================================")
    print("")


A river went out of Eden to water the garden; and from there it was parted, and became four heads.
A rzeka wychodziła z Eden dla odwilżenia sadu; i stamtąd dzieliła się na cztery główne rzeki;

unstyled response: Rzeka wypłynęła z Edenu, aby nawodnić ogród; a stamtąd rozdzieliła się i stała się czterema głowami.
styled response: I wyszedł z ogrodu Eden rzeka, na to aby nawadniała ogród; od onej się zaś rostrowała, i rozpadała się na głów czterech.
Translacja tego zdania pozostaje bez zmian. Zgodnie z prośbą, nie dodam nic więcej.


The name of the first is Pishon: this is the one which flows through the whole land of Havilah, where there is gold;
Imię jednej Fyson; ta okrąża wszystką ziemię Hewila, gdzie się rodzi złoto.

unstyled response: Imię pierwszego to Pishon: jest to ten, który płynie przez całą krainę Havilah, gdzie jest złoto;
styled response: Imię pierwszego Piszon: ten jest, który wpływa przez całą ziemię Chawila, gdzie jest złoto.


and the gold of that land is good. There

In [14]:
witcher_dataset = ["I'm very sorry, Uncle Vesemir.", "But you were asleep, Uncle Vesemir.", "Uncle Vesemir's words."]

In [15]:
for i in range(5):
    prompt = prompt_chat('llama3.1', "Przetłumacz na polski i nie dodawaj niczego więcej: 'I'm very sorry, Uncle Vesemir.'")
    print(prompt)

" Bardzo Ci przykro, Wujku Vesemirze."
Bardzo mi przykro, wujku Vesemirze.
Bardzo mi przykro, wuju Vesemirze.
" Bardzo przepraszam, wujku Vesemie.
Przepraszam bardzo, wujku Vesemirze.


In [16]:
for i in range(5):
    prompt = prompt_chat_custom_temperature('llama3.1', "Przetłumacz na polski i nie dodawaj niczego więcej: 'I'm very sorry, Uncle Vesemir.'", 0.0)
    print(prompt)

" Bardzo mi przykro, wuju Vesemirze."
" Bardzo mi przykro, wuju Vesemirze."
" Bardzo mi przykro, wuju Vesemirze."
" Bardzo mi przykro, wuju Vesemirze."
" Bardzo mi przykro, wuju Vesemirze."


In [17]:
for i in range(5):
    prompt = prompt_chat('llama3.1', "Translate to polish and don't add anything else: 'I'm very sorry, Uncle Vesemir.'")
    print(prompt)

" Bardzo mi przykro, wuju Vesemirze."
Przepraszam bardzo, Wujku Vesimir.
Przepraszam bardzo, Wujku Vesimiru.
Przepraszam bardzo, wuju Vesemir.
Przykro mi bardzo, panie Wieszczu.


In [18]:
for i in range(5):
    prompt = prompt_chat_custom_temperature('llama3.1', "Translate to polish and don't add anything else: 'I'm very sorry, Uncle Vesemir.'", 0.0)
    print(prompt)

Przepraszam bardzo, wuju Wesemirze.
Przepraszam bardzo, wuju Wesemirze.
Przepraszam bardzo, wuju Wesemirze.
Przepraszam bardzo, wuju Wesemirze.
Przepraszam bardzo, wuju Wesemirze.


In [11]:
prompt = prompt_chat('llama3.1', "Znajdź wszystkie nazwy własne w podanym zdaniu i zwróć je oddzielając je przecinkiem. Nie dodawaj nic więcej. 'I'm very sorry, Uncle Vesemir.'")
prompt

'Uncle Vesemir'

In [12]:
prompt = prompt_chat('llama3.1', "Znajdź wszystkie nazwy własne w podanym zdaniu i zwróć je oddzielając je przecinkiem. Nie dodawaj nic więcej. 'I'm very sorry, uncle Vesemir.'")
prompt

'Vesemir'

In [13]:
imie = prompt_chat('llama3.1', "Znajdź wszystkie nazwy własne w podanym zdaniu i zwróć je oddzielając je przecinkiem. Nie dodawaj nic więcej. 'i'm very sorry, uncle vesemir.'")
imie

'Vesemir'

In [14]:
prompt = prompt_chat('llama3.1', "Przetłumacz na polski i nie dodawaj niczego więcej: 'I'm very sorry, Uncle Vesemir.'")
print(prompt)

" Bardzo mi przykro, Wujku Wesemir.


In [15]:
prompt = prompt_chat('llama3.1', f"Oto wszystkie nazwy własne w zdaniu {imie}. Przetłumacz podane zdanie na polski, zachowując wszystkie podane nazwy własne nie zmienione i nie dodawaj nic więcej: 'I'm very sorry, Uncle Vesemir.'")
print(prompt)

Przepraszam bardzo, Wujku Vesemir.


In [16]:
prompt = prompt_chat('llama3.1', f"Oto wszystkie nazwy własne w zdaniu {imie}. Przetłumacz podane zdanie na polski i nie dodawaj nic więcej. Zachowaj wszystkie podane nazwy własne i odmieniaj je tylko poprzez dodanie poprawnych polskich końcówek: 'I'm very sorry, Uncle Vesemir.'")
print(prompt)

Bardzo mi przykro, Wujku Vesemirze.


In [17]:
for i in range(5):
    prompt = prompt_chat('llama3.1', f"Oto wszystkie nazwy własne w zdaniu {imie}. Przetłumacz podane zdanie na polski i nie dodawaj nic więcej. Zachowaj wszystkie podane nazwy własne i odmieniaj je tylko poprzez dodanie poprawnych polskich końcówek: 'I'm very sorry, Uncle Vesemir.'")
    print(prompt)

Bardzo przepraszam Wujku Vesemirze.
Bardzo mi przykro, Wujku Vesemirowi.
Przykro mi, Wujku Wesimirze.
Wujku Vesemiście bardzo mi przykro.
Bardzo przepraszam, wuju Wesemire.


In [18]:
prompt = prompt_chat('llama3.1', f"Wykonaj klasyfikacje NER na podanym zdaniu: 'I'm very sorry, Uncle Vesemir.'")
print(prompt)

W tym zdaniu wykonam klasyfikację NER (Nazwy Jednostek Rzeczowych i Niektóre Ogólne Wyrażenia) następująco:

- "Uncle Vesemir" -> nazwa własna.

Oznaczono że "Uncle Vesemir" jako nazwę własną "Vesemir", a słowo "Uncle" jako angielski odpowiednik polskiego słowa – wuj.


In [19]:
prompt = prompt_chat('llama3.1', f"Wykonaj klasyfikacje NER na podanym zdaniu i zwróć wszystkie nazwy lub imiona własne: 'I'm very sorry, Uncle Vesemir.'")
print(prompt)

Aby wykonać klasyfikację NER (Named Entity Recognition) na tym zdaniu, muszę zidentyfikować części zdania, które są nazwami lub imionami własnymi.

W tym przypadku zdanie brzmi: "I'm very sorry, Uncle Vesemir."

Klasyfikacja wygląda następująco:

- "Uncle" to forma tytułulub stopnia pokrewieństwa, nie jest to jednak same imię czy nazwa własciwa. 
- "Vesemir" uznawane jest za męskie imię i w kontekście zdania pełni rolę zaimienia zwracającego się do rozmówcy.


Zatem jedyną nazwą własną w tym zdaniu jest: **Vesemir** - imię osobowe.


In [20]:
prompt = prompt_chat('llama3.1', f"Wykonaj klasyfikacje NER na podanym zdaniu: 'i'm very sorry, uncle vesemir.'")
print(prompt)

Klasyfikacja Named Entity Recognition (NER) polega na identyfikowaniu w tekście nazwanych jednostek, takich jak imiona, nazwy miejscowości czy państwa. Przeanalizujmy zdanie "i'm very sorry, uncle vesemir."

W tym zdaniu znajduje się jedna nazwana jednostka:

*   "uncle Vesemir" - teoretycznie może być rzeczownikiem oznaczającym stryja, jednak gdy popatrzymy na to bardziej analitycznie można uznać że, "Vesemir" jest to własne imię, a słówko "uncle" jest określeniem rodzaju do osoby o której mowa, więc tutaj właśnie istnieje błąd w prostej identyfikacji i z tego powodu niektórzy zidentyfikują Uncle Vesemira jako jedną nazwaną encję. Po analizie dałoby to jeden wynik:
*   "Uncle Vesemir" - osobę.

Podsumowanie: w danym zdaniu występuje tylko jedna obiekcja, która mieści się w typach NER, ale ze składnią językową możemy przyrównać do dwóch.


In [21]:
prompt = prompt_chat('llama3.1', f"Wykonaj klasyfikacje NER na podanym zdaniu i zwróć wszystkie nazwy własne po przecinku: 'i'm very sorry, uncle vesemir.'")
print(prompt)

Przykładowa odpowiedź to: 'uncle wesemir'.


In [19]:
for i in range(5):
    prompt = prompt_chat_custom_temperature('llama3.1', "Przetłumacz na polski i nie dodawaj niczego więcej: 'geralt, i'm very sorry, uncle vesemir killed lambert.'", 0.0)
    print(prompt)

Geralt, bardzo mi przykro, wujek Vesemir zabił Lamberta.
Geralt, bardzo mi przykro, wujek Vesemir zabił Lamberta.
Geralt, bardzo mi przykro, wujek Vesemir zabił Lamberta.
Geralt, bardzo mi przykro, wujek Vesemir zabił Lamberta.
Geralt, bardzo mi przykro, wujek Vesemir zabił Lamberta.


In [22]:
imiona = prompt_chat('llama3.1', f"Wykonaj klasyfikacje NER na podanym zdaniu i rozpoznaj wszystkie imiona i nazwy własne. Wypisz je i tylko je oddzielając je przecinkiem i nie dodawaj nic więcej: 'geralt, i'm very sorry, uncle vesemir killed lambert.'")
print(imiona)

Geralt, Vesemir, Lambert


In [23]:
for i in range(5):
    prompt = prompt_chat('llama3.1', f"Oto wszystkie nazwy własne w zdaniu {imiona}. Przetłumacz podane zdanie na polski i nie dodawaj nic więcej. Pozostaw wszystkie podane nazwy własne niezmienione: 'geralt, i'm very sorry, uncle vesemir killed lambert.'")
    print(prompt)

Geralt, bardzo mi przykro, wujek Vesemir zabił Lamberta.
Geralt, bardzo mi przykro, stryj Vesemir zabił Lamberta.
Geralt, bardzo mi przykro, stryj Vseimir zabił Lamberta.
Geralt, bardzo mi przykro, wujku  vesemir zabił lamberta.
Geralt, bardzo mi przykro, wujek vesemir zabił lamberta


In [24]:
for a in witcher_dataset:
    print(a)
    imiona1 = prompt_chat('llama3.1', f"Wykonaj klasyfikacje NER na podanym zdaniu i rozpoznaj wszystkie imiona i nazwy własne. Wypisz je i tylko je oddzielając je przecinkiem i nie dodawaj nic więcej: '{a}'")
    print("nazwy własne: ", imiona1)
    for i in range(5):
        prompt = prompt_chat('llama3.1', f"Oto wszystkie nazwy własne w zdaniu {imiona1}. Przetłumacz podane zdanie na polski i nie dodawaj nic więcej. Pozostaw wszystkie podane nazwy własne niezmienione: '{a}'")
        print(prompt)

I'm very sorry, Uncle Vesemir.
nazwy własne:  Vesemir
" Bardzo mi przykro, Wujku Vesemie."
' Bardzo mi przykro, Wujku Vesemir.'
„Bardzo przykro mi Wujku Vesemir”.
Przepraszam bardzo, Wujku Vesemir.
Przepraszam bardzo, wujku Vesemir.
But you were asleep, Uncle Vesemir.
nazwy własne:  Vesemir.
Aleś spał, wuju  Vesimirze.
Aleś spał we śnie, wuju Vesimirze.
Ale ty spałeś, Wujku Vesemir.
Ale spałeś, Wujku Vesimirze.
Ale śpisz, wuju Vesemire.
Uncle Vesemir's words.
nazwy własne:  Vesemir
Wujka Vesemira słowa.
Wujka Vesemira słowa.
Wujek strzelał w słowa Vesemira.
Wujek Vesemir mówił.
Wuja Vesemira słowa.


In [26]:
imiona = prompt_chat('llama3.1', f"Wykonaj klasyfikacje NER na wszystkich wyrazach w podanym zdaniu: 'geralt, i'm very sorry, uncle vesemir killed lambert.'")
imiona

"Klasyfikacja NER (Named Entity Recognition) polega na identyfikacji nazwanych encji w tekście, takich jak imiona, nazwy miejsc czy organizacje. Poniżej przedstawiam wynik klasyfikacji NER dla każdego wyrazu w podanym zdaniu:\n\n* geralt - **OSOBA** (imię własne)\n* I'm - nie jest to jednostka, którą określa się w trakcie rekognicji.\n* very - nie jest to jednostka, którą określa się w trakcie rekognicji.\n* sorry - nie jest to jednostka, którą określa się w trakcie rekognicji.\n* uncle - określenie relacji rodzinnej, lecz nie stanowi odrębnej encji\n* vesemir - **OSOBA** (imię własne)\n* killed - nie jest to jednostka, którą określa się w trakcie rekognicji.\n* lambert - **OSOBA** (imię własne)\n\nWynikiem tej analizy są nastepujące informację:\n- Geralt \n- Vesemir \n- Lambert \n\nSą to imiona własne, które można uznać za nazwane encje."

In [28]:
i_imiona = prompt_chat('llama3.1', f"Oto wykonana klasyfikacja NER: {imiona}, wypisz tylko i wyłącznie wszystkie znalezione nazwy własne i wypisz je po przecinku")
i_imiona

'Geralt, Vesemir, Lambert'

In [29]:
imiona = prompt_chat('llama3.1', f"Wykonaj klasyfikacje NER na wszystkich wyrazach w podanym zdaniu: 'geralt came here one houe ago, killed michael's horse, and took money from lambert'")
i_imiona = prompt_chat('llama3.1', f"Oto wykonana klasyfikacja NER: {imiona}, wypisz tylko i wyłącznie wszystkie znalezione nazwy własne i wypisz je po przecinku")


In [31]:
i_imiona

"Geralt, Michael's-horse, Lambert"

In [32]:
imiona2 = prompt_chat('llama3.1', f"Wykonaj klasyfikacje NER na wszystkich wyrazach w podanym zdaniu: '{i_imiona}'")
i_imiona2 = prompt_chat('llama3.1', f"Oto wykonana klasyfikacja NER: {imiona2}, wypisz tylko i wyłącznie wszystkie znalezione nazwy własne i wypisz je po przecinku")
